# Importing Libraries

In [1]:
import pandas as pd
import csv
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from zipfile import ZipFile
import logging

**Creating Function**

In [2]:
def download_file(save_location,url,file_name):
  
  '''Download any file with file name and url provided'''
  

  local_filename = save_location + file_name
  local_filename = '/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleye'+ file_name
  
  
  # NOTE the stream=True parameter below
  with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(local_filename, 'wb') as f:
         for chunk in r.iter_content(chunk_size=1024): 
            f.write(chunk)
  return (local_filename)

In [3]:
def get_zip_url(xml_file_path):
  
  '''Extract zip file from xml file'''
  
  
  empty_list=[]
  
  f = open(xml_file_path, 'r')
  
  xml_file = f.read()
  
  soup = BeautifulSoup(xml_file, 'lxml')
  
  
  for tag in soup.findAll("str"):
    #print(tag)
    
    if tag["name"] == 'download_link':
      empty_list.append(tag.text)
  return empty_list
  
  f.close()

In [4]:
def create_csv(xml_file, csv_path):
    """ Creates CSV from XML File """
    try:
        # Checking if the path exists or not
        if not os.path.exists(csv_path):
            # Creating the path
            logging.warning("Creating CSV file path")
            os.makedirs(csv_path)

        # Extracting the csv file name from xml file
        csv_fname = xml_file.split(os.sep)[-1].split(".")[0] + ".csv"

        # Creating csv file path
        csv_file = os.path.join(csv_path, csv_fname)

        logging.warning("Loading the xml file")
        # Creating xml file itertor
        xml_iter = ET.iterparse(xml_file, events=("start",))

        csv_columns = [
            "FinInstrmGnlAttrbts.Id",
            "FinInstrmGnlAttrbts.FullNm",
            "FinInstrmGnlAttrbts.ClssfctnTp",
            "FinInstrmGnlAttrbts.CmmdtyDerivInd",
            "FinInstrmGnlAttrbts.NtnlCcy",
            "Issr",
        ]

        # Creating empty dataframe with the required column names
        df = pd.DataFrame(columns=csv_columns)

        # List to store the extacted data
        extracted_data = []

        logging.warning("Parsing the xml file...")
        logging.warning("Extracting the required data from xml")
        # Traversing the xml data
        for event, element in xml_iter:

            # Checking for start of the tags
            if event == "start":

                # Checking for TermntdRcrd tag in which the required data is
                if "TermntdRcrd" in element.tag:

                    # Dictionary to store require data in single element
                    data = {}

                    # List of the required tags (FinInstrmGnlAttrbts, Issr)
                    reqd_elements = [
                        (elem.tag, elem)
                        for elem in element
                        if "FinInstrmGnlAttrbts" in elem.tag or "Issr" in elem.tag
                    ]

                    # Traversing through the required tags
                    for tag, elem in reqd_elements:

                        if "FinInstrmGnlAttrbts" in tag:

                            # Traversing through the child elements of
                            # FinInstrmGnlAttrbts element
                            for child in elem:

                                # Adding the extrcated data in the dictionary
                                if "Id" in child.tag:
                                    data[csv_columns[0]] = child.text
                                elif "FullNm" in child.tag:
                                    data[csv_columns[1]] = child.text
                                elif "ClssfctnTp" in child.tag:
                                    data[csv_columns[2]] = child.text
                                elif "CmmdtyDerivInd" in child.tag:
                                    data[csv_columns[3]] = child.text
                                elif "NtnlCcy" in child.tag:
                                    data[csv_columns[4]] = child.text

                        # Extracting Issr Tag value
                        else:
                            data[csv_columns[5]] = child.text

                    # Appending the single element extracted data in the list
                    extracted_data.append(data)

        logging.warning("All the required data extracted from xml file")

        # Appending the extracted data in the data frame
        df = df.append(extracted_data, ignore_index=True)

        logging.warning("Dropping empty rows")
        # Removes empty rows from the dataframe
        df.dropna(inplace=True)

        logging.warning("Creating the CSV file")
        # Creates csv file from the dataframe
        df.to_csv(csv_file, index=False)

        # returning the csv file path
        return csv_file

    except Exception as e:
        print(f"Error occurred while extracting - {str(e)}")

In [5]:
def extract_zip(url_zip,save_location):
  '''function to extract zip file to a directory'''


  # specifying the zip file name
  file_name = url_zip
  loc = '/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleye'
  
  # opening the zip file in READ mode
  with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
    
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall(path=loc)
    print('Done!')

## Implementing Functions

In [6]:
url_path='https://registers.esma.europa.eu/solr/esma_registers_firds_files/select?q=*&fq=publication_date:%5B2021-01-17T00:00:00Z+TO+2021-01-19T23:59:59Z%5D&wt=xml&indent=true&start=0&rows=100'

In [7]:
save_location = '/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleye'

In [8]:
xml_loc = save_location + 'xml_file.xml'

In [9]:
download_file(save_location,url_path,'xml_file.xml')

'/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleyexml_file.xml'

In [10]:
zip_url_list = get_zip_url(xml_loc)

C:\Users\rahul\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [11]:
zip_url_list

['http://firds.esma.europa.eu/firds/DLTINS_20210117_01of01.zip',
 'http://firds.esma.europa.eu/firds/DLTINS_20210119_01of02.zip',
 'http://firds.esma.europa.eu/firds/DLTINS_20210119_02of02.zip',
 'http://firds.esma.europa.eu/firds/DLTINS_20210118_01of01.zip']

In [12]:
for i in range(len(zip_url_list)):
  download_file(save_location,zip_url_list[i],'zipfile'+str(i))

In [13]:
loc = '/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleye'
for i in range(4):
  extract_zip(loc +'zipfile'+str(i),save_location)

File Name                                             Modified             Size
DLTINS_20210117_01of01.xml                     2021-01-17 01:17:12    143278061
Extracting all the files now...
Done!
File Name                                             Modified             Size
DLTINS_20210119_01of02.xml                     2021-01-19 03:21:08    521464972
Extracting all the files now...
Done!
File Name                                             Modified             Size
DLTINS_20210119_02of02.xml                     2021-01-19 03:21:56    363082618
Extracting all the files now...
Done!
File Name                                             Modified             Size
DLTINS_20210118_01of01.xml                     2021-01-18 01:21:22      1439631
Extracting all the files now...
Done!


In [18]:
#input_extracted_xml consist list of xml files extracted from zip


input_extracted_xml=["/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleye/DLTINS_20210117_01of01.xml",
                     "/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleye/DLTINS_20210118_01of01.xml",
                     "/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleye/DLTINS_20210119_01of02.xml",
                     "/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleye/DLTINS_20210119_02of02.xml"]


#output_path_converted_csv is path where converted csv wil be saved



output_path_converted_csv="/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleye/"

In [19]:
import os

In [21]:
# for loop to covert all xml files extracted from zip

for link in input_extracted_xml:
  print(link)

  #calling create_csv function to convert xml to csv
  
  create_csv(link, output_path_converted_csv)

/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleye/DLTINS_20210117_01of01.xml


C:\Users\rahul\AppData\Local\Temp\ipykernel_5412\3209183778.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(extracted_data, ignore_index=True)


/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleye/DLTINS_20210118_01of01.xml


C:\Users\rahul\AppData\Local\Temp\ipykernel_5412\3209183778.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(extracted_data, ignore_index=True)


/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleye/DLTINS_20210119_01of02.xml


C:\Users\rahul\AppData\Local\Temp\ipykernel_5412\3209183778.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(extracted_data, ignore_index=True)


/Users/rahul/OneDrive/Desktop/jupyter Projects/Steeleye/steeleye/DLTINS_20210119_02of02.xml


C:\Users\rahul\AppData\Local\Temp\ipykernel_5412\3209183778.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(extracted_data, ignore_index=True)


In [22]:
df=pd.read_csv('/Users/rahul/OneDrive/Desktop/Jupyter Projects/Steeleye/steeleye/DLTINS_20210117_01of01.csv')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100086 entries, 0 to 100085
Data columns (total 6 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   FinInstrmGnlAttrbts.Id              100086 non-null  object
 1   FinInstrmGnlAttrbts.FullNm          100086 non-null  object
 2   FinInstrmGnlAttrbts.ClssfctnTp      100086 non-null  object
 3   FinInstrmGnlAttrbts.CmmdtyDerivInd  100086 non-null  bool  
 4   FinInstrmGnlAttrbts.NtnlCcy         100086 non-null  object
 5   Issr                                100086 non-null  bool  
dtypes: bool(2), object(4)
memory usage: 3.2+ MB


In [24]:
df.head()

,FinInstrmGnlAttrbts.Id,FinInstrmGnlAttrbts.FullNm,FinInstrmGnlAttrbts.ClssfctnTp,FinInstrmGnlAttrbts.CmmdtyDerivInd,FinInstrmGnlAttrbts.NtnlCcy,Issr
0,DE000A1R07V3,Kreditanst.f.Wiederaufbau Anl.v.2014 (2021),DBFTFB,False,EUR,False
1,DE000A1R07V3,KFW 1 5/8 01/15/21,DBFTFB,False,EUR,False
2,DE000A1R07V3,Kreditanst.f.Wiederaufbau Anl.v.2014 (2021),DBFTFB,False,EUR,False
3,DE000A1R07V3,Kreditanst.f.Wiederaufbau Anl.v.2014 (2021),DBFTFB,False,EUR,False
4,DE000A1X3J56,IKB Deutsche Industriebank AG Stufenz.MTN-IHS ...,DTVUFB,False,EUR,False


In [25]:
from io import StringIO 

import boto3


#Creating Session With Boto3.
session = boto3.Session(
aws_access_key_id='XXXXXX',
aws_secret_access_key='XXXXXX'
)

#Creating S3 Resource From the Session.
s3_res = session.resource('s3')

csv_buffer = StringIO()

df.to_csv(csv_buffer)

bucket_name = 'assignmentsteeleye'

s3_object_name = 'DLTINS_20210117_01of01.csv'

s3_res.Object(bucket_name, s3_object_name).put(Body=csv_buffer.getvalue())

print("Dataframe is saved as CSV in S3 bucket.")

Dataframe is saved as CSV in S3 bucket.
